In [1]:
from phe.util import getprimeover, invert, powmod
import numpy as np
import secrets

# [pip/conda conda install numpy phe gmpy2]

Módulo com a nossa implementação do Paillier Cryptosystem.

Contém as gerações de chaves, encriptação e desencriptação, e as operações garantidas pelas propriedades homomórficas do sistema.

In [2]:
def L(x, n):
    return (x - 1) // n

class PublicKey:
    def __init__(self, n, g):
        self.n = n
        self.g = g

    def encrypt(self, m):
        n = self.n
        g = self.g
        r = 0
        while r == 0:
            r = secrets.randbelow(self.n)
        
        return (powmod(g, m, n**2)) * (powmod(r, n, n**2)) % n**2
    
    #calcula o ciphertext da soma dos dois plaintexts encriptados em ctext1 e ctext2
    def add(self, ctext1, ctext2):
        return (ctext1 * ctext2) % self.n**2
    
    #calcula o ciphertext do produto dos plaintext em ptext e do plaintext encriptado em ctext
    def mul(self, ctext, ptext):
        return powmod(ctext, ptext, self.n**2)
    
    #o número aleatório 'presente' na enriptação de um número deve ser re-gerado após somas ou multiplicações
    def obfuscate(self, c):
        n = self.n
        r = 0
        while r == 0:
            r = secrets.randbelow(self.n)

        return c * (powmod(r, n, n**2)) % n**2


class PrivateKey:
    def __init__(self, lam, mu, pk):
        self.lam = lam
        self.mu = mu
        self.pk = pk

    def decrypt(self, c):
        lam = self.lam
        mu = self.mu
        n = self.pk.n

        return ((L(powmod(c, lam, n**2), n) % n) * mu) % n


def generateKeyPair(n_length):
    p = q = n = None
    n_len = 0
    while n_len != n_length:
        p = getprimeover(n_length // 2)
        q = getprimeover(n_length // 2)
        n = p * q
        n_len = n.bit_length()

    g = n + 1
    phi_n = (p - 1) * (q - 1)
    lam = phi_n
    mu = invert(phi_n, n)

    pk = PublicKey(n, g)
    sk = PrivateKey(lam, mu, pk)

    return pk, sk

Vale a pena notar que não foi implementado nenhum sistema de (n, n)-threshold decryption, uma vez que considerámos que ficava bastante fora do âmbito do projeto. A encriptação fica portanto, de certo modo, inútil, uma vez que cada jogador tem acesso à chave privada com que pode desencriptar qualquer polinómio que recebe a qualquer momento. Mesmo assim, achamos que o projeto mostra a implementação do protocolo, e mostra que este funciona mesmo quando a informação trocada e processada (como nas somas e multiplicações de ciphertexts) está encriptada através do sistema criptográfico relevante.

Módulo com a nossa implementação de polinómios.
As operações de soma e multiplicação de polinómios estão definidas tanto quando os coeficientes são inteiros como quando os coeficientes de um polinómio estão encriptados.

In [3]:
class MyPolynomial:
    def __init__(self, coef):
        self.coef = []
        for x in coef:
            self.coef.append(int(x))

    def __call__(self, x, n):
        y = 0
        for i in range(self.n_coef()):
            y += (self.coef[i] * powmod(x, i, n)) % n
            y %= n
        return y

    def empty(n):
        return MyPolynomial(np.zeros(n))

    def fromroots(roots):
        new_poly = MyPolynomial([1])
        for x in roots:
            new_poly = new_poly.mul(MyPolynomial([-x, 1]))
        return new_poly

    def n_coef(self):
        return len(self.coef)
    
    def degree(self):
        return len(self.coef) - 1

    def reduce(self):
        last = self.n_coef()
        for i in range(len(self.coef)-1, -1, -1):
            if self.coef[i] != 0:
                break
            else:
                last -= 1
        self.coef = self.coef[:last]


    def add(self, poly):
        if self.n_coef() > poly.n_coef():
            sum = self.coef
            for i in range(poly.n_coef()):
                sum[i] += poly.coef[i]
            new_poly = MyPolynomial(sum)
        else:
            sum = poly.coef
            for i in range(self.n_coef()):
                sum[i] += self.coef[i]
            new_poly = MyPolynomial(sum)
        new_poly.reduce()
        return new_poly
    
    #adição quando os dois polinómios têm os coeficientes encriptados
    def add_paillier(self, poly, pk):
        if self.n_coef() > poly.n_coef():
            sum = self.coef
            for i in range(poly.n_coef()):
                sum[i] = pk.add(self.coef[i], poly.coef[i])
            new_poly = MyPolynomial(sum)
        else:
            sum = poly.coef
            for i in range(self.n_coef()):
                sum[i] = pk.add(self.coef[i], poly.coef[i])
            new_poly = MyPolynomial(sum)
        new_poly.reduce()
        return new_poly
    
    def mul(self, poly):
        new_coef = self.n_coef() + poly.n_coef() - 1
        new_poly = MyPolynomial.empty(new_coef)

        for i in range(self.n_coef()):
            for j in range(poly.n_coef()):
                new_poly.coef[i+j] += self.coef[i] * poly.coef[j]
        new_poly.reduce()
        return new_poly
    
    #multiplicação quando um dos polinómios (self) tem os coeficientes encriptados
    def mul_paillier(self, poly, pk):
        new_coef = self.n_coef() + poly.n_coef() - 1
        new_poly = MyPolynomial.empty(new_coef)
        new_poly = MyPolynomial([pk.encrypt(x) for x in new_poly.coef])

        for i in range(self.n_coef()):
            for j in range(poly.n_coef()):
                new_poly.coef[i+j] = pk.add(new_poly.coef[i+j], pk.mul(self.coef[i], poly.coef[j]))
        new_poly.reduce()
        return new_poly
    
    def deriv(self):
        if self.n_coef() <= 1:
            return MyPolynomial([0])    

        new_poly = MyPolynomial.empty(self.n_coef() - 1)
        for i in range(new_poly.n_coef()):
            new_poly.coef[i] = self.coef[i+1] * (i + 1)
        new_poly.reduce()
        return new_poly

Classe correspondente a um jogador no protocolo.

In [4]:
class Player:

    def __init__(self, S, sk):
        self.S = S  #multiset S_i a intersetar
        self.k = len(S)
        self.sk = sk    #chave secreta
        self.pk = sk.pk
        self.f_list = []
        self.N = self.pk.n
    
    def polynomial(self):
        # cria o polinómio f a partir do conjunto S 
        self.f = MyPolynomial.fromroots(self.S)
    
    def encrypt(self):
        # encripta o polinómio f com pk
        self.ef = MyPolynomial([self.pk.encrypt(x) for x in self.f.coef])
        self.f_list.append(self.ef)
    
    def append_f(self, encrypted_polynomial):
        # adiciona o polinomio encriptado de outro jogador a f_list
        self.f_list.append(encrypted_polynomial)

    def receive_l(self, encrypted_polynomial):
        # recebe lambda_i-1
        self.lam_im1 = encrypted_polynomial
    
    def receive_p(self, encrypted_polynomial):
        #recebe p = lambda_n
        self.ep = encrypted_polynomial

    def phi(self):
        # calcula phi_i
        c = len(self.f_list)
        r = []

        #escolhe os r_i,js uniformemente
        for _ in range(c):
            r.append(MyPolynomial([secrets.randbelow(self.N) for _ in range(self.k + 1)]))

        sum = MyPolynomial([self.pk.encrypt(0)])
        for j in range(c):
            prod = self.f_list[j].mul_paillier(r[j], self.pk)
            sum = sum.add_paillier(prod, self.pk)
        self.phi_i = sum
        
        #re-randomização dos coeficientes
        #esta operação é feita uma única vez no fim das contas todas, uma vez que é cara e ninguém tem acesso aos valores intermédios
        for i in range(len(self.phi_i.coef)):
            self.phi_i.coef[i] = self.pk.obfuscate(self.phi_i.coef[i])

    def lam(self):
        # calcula lambda_i
        self.lam_i = self.lam_im1.add_paillier(self.phi_i, self.pk)
        
        for i in range(len(self.lam_i.coef)):
            self.lam_i.coef[i] = self.pk.obfuscate(self.lam_i.coef[i]) #re-randomização dos coeficientes

    
    def decrypt(self):
        self.p = MyPolynomial([self.sk.decrypt(x) for x in self.ep.coef])

    def multiset(self):
        # calcula o multiset de intersecção na forma [(a,b),...,(a,b)], a \in S_i
        # onde o elemento a aparece b vezes no multiset
        # para encontrar o grau de uma raíz a em p, vamos derivando p e avaliando p(a)

        res = []
        S_new = list(dict.fromkeys(self.S))

        for elem in S_new:
            degree = 0
            poly = self.p
            while poly(elem, self.N) == 0:
                degree += 1
                poly = poly.deriv()
            if degree > 0:
                res.append((elem, degree))
        self.intersection = res

    

Implementação do protocolo (os números e letras nos comentários seguem o esquema da secção 5.1 do artigo)

In [5]:
def psi(players, c):
    """
    Perform a private Set-Intersection secure against a coalition of honest-but-curious
    adversaries.

    Parameters
    ----------
    players : array_like construct of class Player elements
    c : dishonestly colluding constant
    """
    n = len(players)
    assert n >= 2, "Not enough players"
    assert c < n, "Dishonestly colluding too large"
    assert all([player.k == players[0].k for player in players]),\
        "Differently sized private sets"
    #1
    for player in players:
        #a
        player.polynomial()       
        player.encrypt()
    
    #b
    for i in range(n):
        for j in range(1, c+1):
            players[i].append_f(players[(i-j) % n].ef)
        
        #c, d
        players[i].phi()

    #2
    players[0].lam_i = players[0].phi_i
    players[1].receive_l(players[0].lam_i)

    #3
    for i in range(1, n):
        #a é realizado implicitamente
        #b
        players[i].lam()

        #c
        players[(i+1)%len(players)].receive_l(players[i].lam_i)
    #4
    for player in players:
        player.receive_p(players[0].lam_im1)

    #5
    for player in players:
        player.decrypt()
        pass
    #6
    for player in players:
        player.multiset()

Um exemplo de execução:

In [6]:
pk, sk = generateKeyPair(1024)
print("Chave gerada")

a = Player([1,1,2,4,3], sk)
b = Player([1,1,1,2,1], sk)
c = Player([1,1,2,6,1], sk)
d = Player([1,6,3,1,0], sk)
e = Player([1,1,3,4,6], sk)

psi([a,b,c,d,e], 3)
print("Interseção completa")

Chave gerada
Interseção completa


In [7]:
#Imprimir resultado no formato (elemento, número de occorências)
for x in a.intersection:
    print(x)

(1, 2)


In [8]:
a = Player([1,1,2,3,4], sk)
b = Player([5,6,1,2,1], sk)
c = Player([9,8,7,6,6], sk)


psi([a,b,c], 2)
print("Interseção completa")

#Imprimir resultado no formato (elemento, número de occorências)
for x in a.intersection:
    print(x)

Interseção completa


In [9]:
a = Player([1,1,2,3,4,6,6,6,6,6,6,6,6,7,7,7,7,7,7,7,7,7,8,8,8,8,8,8,8,8,1,1,1,1,1,1,1], sk)
b = Player([5,6,1,2,1,1,1,1,1,1,1,5,5,5,5,5,5,5,5,6,6,6,6,6,6,6,6,6,2,2,2,2,2,2,2,2,2], sk)
c = Player([9,8,7,6,6,6,6,6,6,6,6,6,6,1,1,1,1,1,1,1,1,1,7,7,7,7,7,7,7,8,8,8,8,8,8,8,8], sk)


psi([a,b,c], 1)
print("Interseção completa")

#Imprimir resultado no formato (elemento, número de occorências)
for x in a.intersection:
    print(x)

Interseção completa
(1, 8)
(6, 8)
